In [ ]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from keras.applications.densenet import DenseNet169
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images_path = {}
images_path["Covid"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.png")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpeg")
images_path["Covid"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/Covid-19/*.jpg")

images_path["NoFindings"] = glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.png")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpeg")
images_path["NoFindings"] += glob("/content/drive/MyDrive/X-Ray Image DataSet/No_findings/*.jpg")

images_class = {
    "Covid": 0,
    "NoFindings": 1
}

In [ ]:
X = []
Y = []

for label in images_path:
    for image_path in images_path[label]:
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224, 224))
        X.append(image)
        Y.append(images_class[label])

In [ ]:
X = np.array(X)
Y = np.array(Y)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, Y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train=Y[train_index]
    y_test=Y[test_index]
X_train=np.array(X_train).reshape(-1,224,224,3).astype('float32')
y_train=np.array(y_train)
X_test=np.array(X_test).reshape(-1,224,224,3).astype('float32')
y_test=np.array(y_test)

In [ ]:
X_train=tf.keras.applications.densenet.preprocess_input(X_train)
X_test=tf.keras.applications.densenet.preprocess_input(X_test)

In [ ]:
densenet.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                        

In [ ]:
densenet=tf.keras.applications.DenseNet169(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [ ]:
densenet.trainable = False
for layer in densenet.layers:
     layer.trainable = False

In [ ]:
base_model_output = densenet.output
gap = keras.layers.GlobalAveragePooling2D()(base_model_output)
fc3=keras.layers.Dense(units=512, activation="relu",kernel_regularizer='l2')(gap)
BN3=tf.keras.layers.BatchNormalization()(fc3)
output = keras.layers.Dense(1, activation="sigmoid")(BN3)

In [ ]:
transfer_model = keras.models.Model(densenet.input, output)

In [ ]:
transfer_model.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,epochs=20,batch_size=8,validation_split=0.1)

Epoch 1/20
57/57 [==============================] - 12s 74ms/step - loss: 4.1105 - accuracy: 0.8022 - val_loss: 2.7683 - val_accuracy: 0.5400
Epoch 2/20
57/57 [==============================] - 2s 35ms/step - loss: 1.6352 - accuracy: 0.9356 - val_loss: 1.2507 - val_accuracy: 0.9800
Epoch 3/20
57/57 [==============================] - 2s 35ms/step - loss: 1.1071 - accuracy: 0.9622 - val_loss: 0.8568 - val_accuracy: 1.0000
Epoch 4/20
57/57 [==============================] - 2s 35ms/step - loss: 0.7920 - accuracy: 0.9644 - val_loss: 0.8010 - val_accuracy: 0.9000
Epoch 5/20
57/57 [==============================] - 2s 35ms/step - loss: 0.6150 - accuracy: 0.9756 - val_loss: 0.5696 - val_accuracy: 0.9600
Epoch 6/20
57/57 [==============================] - 2s 35ms/step - loss: 0.4927 - accuracy: 0.9667 - val_loss: 1.8764 - val_accuracy: 0.5400
Epoch 7/20
57/57 [==============================] - 2s 35ms/step - loss: 0.4466 - accuracy: 0.9711 - val_loss: 1.8798 - val_accuracy: 0.5000
Epoch 8/20
5

In [ ]:
densenet.trainable = True
for layer in densenet.layers:
     layer.trainable = True

In [ ]:
ADAM= keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(loss="binary_crossentropy",optimizer=ADAM,metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,batch_size=8,epochs=10,validation_split=0.1)

Epoch 1/10
57/57 [==============================] - 5s 90ms/step - loss: 0.1033 - accuracy: 1.0000 - val_loss: 0.0991 - val_accuracy: 1.0000
Epoch 2/10
57/57 [==============================] - 5s 90ms/step - loss: 0.1083 - accuracy: 0.9956 - val_loss: 0.0963 - val_accuracy: 1.0000
Epoch 3/10
57/57 [==============================] - 5s 90ms/step - loss: 0.1107 - accuracy: 0.9956 - val_loss: 0.1024 - val_accuracy: 1.0000
Epoch 4/10
57/57 [==============================] - 5s 90ms/step - loss: 0.1159 - accuracy: 0.9933 - val_loss: 0.1070 - val_accuracy: 0.9800
Epoch 5/10
57/57 [==============================] - 5s 90ms/step - loss: 0.0981 - accuracy: 0.9956 - val_loss: 0.0916 - val_accuracy: 1.0000
Epoch 6/10
57/57 [==============================] - 5s 90ms/step - loss: 0.1009 - accuracy: 0.9956 - val_loss: 0.0902 - val_accuracy: 1.0000
Epoch 7/10
57/57 [==============================] - 5s 90ms/step - loss: 0.0854 - accuracy: 1.0000 - val_loss: 0.0875 - val_accuracy: 1.0000
Epoch 8/10
57

In [ ]:
mse_test = transfer_model.evaluate(X_test, y_test)

4/4 [==============================] - 1s 254ms/step - loss: 0.1181 - accuracy: 0.9760
